In [15]:
%run -i opt_model.py

In [16]:
m

<__main__.Manager instance at 0x105a63b48>

In [17]:
m.run_till(100)

In [18]:
m.state.time

99.6006013457742

In [20]:
sum(1 for ev in m.state.sinks[1000] if ev.src_id == 1)

97

In [21]:
sum(1 for ev in m.state.sinks[1000] if ev.src_id == 2)

101